In [1]:
import os
import torch
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from feature_eng.scalers import ranged_scaler
from datetime import datetime, timedelta
#from mpge.rca import mpge_root_cause_diagnosis
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# Or if you are using > Python 3.11:
with warnings.catch_warnings(action="ignore"):
    fxn()

In [2]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [3]:
from tkgngc.embeddings import PretrainedTKGEmbeddingWithTimestamps
from tkgngc.data_processing import TKGNGCDataProcessor
from tkgngc.model import train_model

In [4]:
cats_df = pl.read_csv("data/data.csv", separator=",")  
metadata = pl.read_csv('data/metadata.csv',separator=',')
potential_causes = metadata['root_cause'].unique().to_list()

for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

In [5]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.Series("entity_id",range(len(cats_df)))
)

In [6]:
cats_rows_list = metadata.rows(named=True)
cats_df.head()

timestamp,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1,y,category,entity_id
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
2023-01-01 00:00:00,0.0,0.571429,0.25,0.5,0.5,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688,0.0,0.0,0
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.40973,0.246024,0.14197,0.112819,0.550194,0.406691,0.0,0.0,1
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698,0.0,0.0,2
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.40671,0.0,0.0,3
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726,0.0,0.0,4


In [7]:
cats_df = cats_df.to_pandas()

In [8]:
potential_causes = metadata['root_cause'].unique().to_list()

In [9]:
cats_df=cats_df.set_index('timestamp')
cats_df = cats_df.drop(['y','category','entity_id'],axis=1)
cats_df.head()


,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,0.0,0.571429,0.250000,0.50000,0.5000,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409730,0.246024,0.14197,0.112819,0.550194,0.406691
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.406710
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726


In [10]:
train_df = cats_df[0:1000000]
test_df = cats_df[1000000:]

In [11]:
train_df.head()

,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
timestamp,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,0.0,0.571429,0.250000,0.50000,0.5000,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409726,0.246024,0.14197,0.112819,0.550194,0.406688
2023-01-01 00:00:01,0.0,0.571429,0.252001,0.50001,0.5001,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409730,0.246024,0.14197,0.112819,0.550194,0.406691
2023-01-01 00:00:02,0.0,0.571429,0.256914,0.50002,0.5002,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409741,0.246024,0.14197,0.112819,0.550195,0.406698
2023-01-01 00:00:03,0.0,0.571429,0.268273,0.50003,0.5003,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409758,0.246024,0.14197,0.112819,0.550195,0.406710
2023-01-01 00:00:04,0.0,0.571429,0.277953,0.50004,0.5004,0.0,0.0,0.33599,0.315382,0.130919,0.116409,0.409782,0.246024,0.14197,0.112819,0.550197,0.406726


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")


In [13]:
device

device(type='cuda')

In [14]:
train_df.columns

Index(['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1'],
      dtype='object')

In [15]:
tkgnc_data = TKGNGCDataProcessor(train_df,device,num_timestamps=20, lags=1)

In [16]:
pretrained_tkg = PretrainedTKGEmbeddingWithTimestamps(
    num_entities=int(tkgnc_data.entity_indices.max().item()+1),
    num_relations=int(tkgnc_data.relation_indices.max().item()+1),
    embedding_dim=17,
    num_timestamps=tkgnc_data.num_timestamps,
).to(device)

In [17]:
quads = (
    tkgnc_data.entity_indices[:-1],  # Head entities
    tkgnc_data.relation_indices,  # Relations
    tkgnc_data.entity_indices[1:],  # Tail entities (shifted example)
    tkgnc_data.timestamp_indices[:-1],  # Timestamps
)


In [18]:
#pretrained_tkg.pretrain(quads, learning_rate=0.01, epochs=1000)
#torch.save(pretrained_tkg.state_dict(), 'pretrained_tkg')
pretrained_tkg.load_state_dict(torch.load('pretrained_tkg'))

<All keys matched successfully>

In [19]:
from CARAT.data_processing import *
from CARAT.model import *
from CARAT.causal_inference import causal_inference

Using device: cuda


C:\Users\jlowh\anaconda3\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
try:
    train_df = train_df.drop('time',axis=1)
except:
    None
try:
    test_df = test_df.drop('time',axis=1)
except:
    None

In [21]:
cols = list(train_df.columns)
non_causal_columns = list(set(cols).difference(set(potential_causes)))
causal_indices = [train_df.columns.get_loc(col) for col in potential_causes]
non_causal_indices = [train_df.columns.get_loc(col) for col in non_causal_columns]

num_nodes = len(train_df.columns)

new_metadata = []
A0 = get_adjacency(cols,causal_indices,non_causal_indices,num_nodes)    
BATCH_SIZE = 64

model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [22]:
from utils.utils import set_seed
set_seed()
n_correct = 0
total_checked = 0
for i, row in enumerate(cats_rows_list):
    total_checked +=1 
    print('Model: '+ str(i))
    anomaly = eval(row['affected'])[0]

    anomaly_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    #start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    root_cause = row['root_cause']
    mod_df = test_df[(test_df.index>= anomaly_time) & (test_df.index<= end_time)]
    mod_df = mod_df[['aimp', 'amud', 'arnd', 'asin1', 'asin2', 'adbr', 'adfl', 'bed1',
       'bed2', 'bfo1', 'bfo2', 'bso1', 'bso2', 'bso3', 'ced1', 'cfo1', 'cso1']]
    num_nodes = len(mod_df.columns)

    start_len = mod_df.shape[0]
    if start_len >1000:
        start_len = 1000

    start_time = anomaly_time- timedelta(seconds=start_len)

    normal_df = test_df[(test_df.index>= start_time) & (test_df.index< anomaly_time)]

    sample_data = TKGNGCDataProcessor(mod_df,device,num_timestamps=20, lags=1)
    model_data = create_granger_gat_data(pretrained_tkg,sample_data)
    model_data.retrain_tkg()
    features = model_data.time_series_data
    model_name = 'model_category_'+str(row['category'])
    """model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))"""
    model = CausalGraphVAE(input_dim=num_nodes, hidden_dim=128,
                   latent_dim=16, 
                   num_nodes=num_nodes, 
                   embed_dim=17,
                   prior_adj_matrix=A0.to(torch.float))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    data = create_lagged_features(features, 3, pad_value=0)
    ee = create_lagged_features(model_data.entity_emb, 3, pad_value=0)
    tt = create_lagged_features(model_data.timestamp_emb, 3, pad_value=0)
        
    dataset = TimeSeriesDataset(data, ee, tt)
    dataloader = DataLoader(dataset,batch_size = BATCH_SIZE, shuffle=False)
    loss = train_causal_vae(model, optimizer, dataloader, A0, num_epochs=1000)

    model.eval()
    with torch.no_grad():
        _, _, _, learned_adj_matrix = model(data,ee,tt, num_nodes=num_nodes)
        causal_graph = learned_adj_matrix.cpu().numpy()


    adj_data = pd.DataFrame(causal_graph,index=cols,columns=cols)
    candidates = adj_data[anomaly].sort_values(ascending=False)
    candidates = candidates[candidates.index.isin( potential_causes)]
    #A0 = learned_adj_matrix
    try:
        ci = causal_inference(causal_graph, mod_df, normal_df,potential_causes)
        ci.infer_causal_path(anomaly)  
    
        if len(ci.causal_factors) >= 3:
            potential_cause1 = ci.causal_factors[0][0]
            potential_cause2 = ci.causal_factors[1][0]
            potential_cause3 = ci.causal_factors[2][0]
        elif len(ci.causal_factors) == 2:
            potential_cause1 = ci.causal_factors[0][0]
            potential_cause2 = ci.causal_factors[1][0]
            potential_cause3 = 'NA'
        elif len(ci.causal_factors) == 1:
            potential_cause1 = ci.causal_factors[0][0]
            potential_cause2 = "NA"
            potential_cause3 = 'NA'
        else:
            potential_cause1 = candidates.index[0]
            potential_cause2 = candidates.index[1]
            potential_cause3 = candidates.index[2]
    except:
        potential_cause1 = candidates.index[0]
        potential_cause2 = candidates.index[1]
        potential_cause3 = candidates.index[2]

   
    if root_cause in [potential_cause1,potential_cause2,potential_cause3]:
        n_correct+=1

    if root_cause == potential_cause1:
        row['cause_1'] = 1
    if root_cause == potential_cause2:
        row['cause_2'] = 1
    if root_cause == potential_cause3:
        row['cause_3'] = 1
    new_metadata.append(row)
    if root_cause in [potential_cause1 , potential_cause2 , potential_cause3]:
        print('ROOT CAUSE FOUND!')
    print(root_cause + '-->' + potential_cause1 + ' | ' + potential_cause2 + ' | ' + potential_cause3)

    print("Model Accuracy: " + str(round( (n_correct/total_checked)*100,2)) + '%' )
    
    print('--------------------------------------------------------------------------------------------------------------------------------')



Model: 0
Epoch 0, Loss: 0.0011
Epoch 0: Average Loss = 226.6278, Best Loss = 226.6278
Epoch 100: Average Loss = 38.9298, Best Loss = 38.7842
Epoch 200: Average Loss = 34.1131, Best Loss = 34.1131
Epoch 300: Average Loss = 32.4063, Best Loss = 32.3149
Early stopping triggered after 383 epochs. Restoring best model.


Evaluating set functions...: 100%|██████████| 321/321 [00:34<00:00,  9.27it/s]


ROOT CAUSE FOUND!
bso3-->bfo1 | bso3 | bfo2
Model Accuracy: 100.0%
--------------------------------------------------------------------------------------------------------------------------------
Model: 1
Epoch 0, Loss: 0.0015
Epoch 0: Average Loss = 186.2172, Best Loss = 186.2172
Epoch 100: Average Loss = 33.4662, Best Loss = 33.4611
Early stopping triggered after 197 epochs. Restoring best model.


Evaluating set functions...: 100%|██████████| 120/120 [00:02<00:00, 59.35it/s]


bso3-->bfo1 | bso1 | bed1
Model Accuracy: 50.0%
--------------------------------------------------------------------------------------------------------------------------------
Model: 2
Epoch 0, Loss: 0.0005
Epoch 0: Average Loss = 281.0646, Best Loss = 281.0646
Epoch 100: Average Loss = 33.4065, Best Loss = 33.4065
Epoch 200: Average Loss = 30.7679, Best Loss = 30.7679
Early stopping triggered after 284 epochs. Restoring best model.


...

bfo2-->bso3 | bso2 | bed1
Model Accuracy: 49.49%
--------------------------------------------------------------------------------------------------------------------------------
Model: 198
Epoch 0, Loss: 0.0001
Epoch 0: Average Loss = 211.5256, Best Loss = 211.5256
Epoch 100: Average Loss = 35.4427, Best Loss = 35.3885
Epoch 200: Average Loss = 33.1341, Best Loss = 33.1341
Epoch 300: Average Loss = 32.5816, Best Loss = 32.5192
Early stopping triggered after 301 epochs. Restoring best model.


Evaluating set functions...: 100%|██████████| 293/293 [00:25<00:00, 11.45it/s]


ROOT CAUSE FOUND!
bfo1-->bso1 | bfo2 | bfo1
Model Accuracy: 49.75%
--------------------------------------------------------------------------------------------------------------------------------
Model: 199
Epoch 0, Loss: 0.0000
Epoch 0: Average Loss = 191.2427, Best Loss = 191.2427
Epoch 100: Average Loss = 33.5877, Best Loss = 33.5877
Epoch 200: Average Loss = 29.7445, Best Loss = 29.7445
Epoch 300: Average Loss = 28.4789, Best Loss = 28.4789
Early stopping triggered after 374 epochs. Restoring best model.


Evaluating set functions...: 100%|██████████| 263/263 [00:21<00:00, 12.39it/s]


ROOT CAUSE FOUND!
bfo2-->bfo2 | bso1 | bso3
Model Accuracy: 50.0%
--------------------------------------------------------------------------------------------------------------------------------


In [25]:
model.eval()
with torch.no_grad():
    _, _, _, learned_adj_matrix = model(data,ee,tt, num_nodes=num_nodes)
    causal_graph = learned_adj_matrix.cpu().numpy()


In [26]:
learned_adj_matrix.shape

torch.Size([17, 17])

In [27]:
test = learned_adj_matrix.view(17* 17)
learned_adj_matrix = learned_adj_matrix.fill_diagonal_(0)

In [30]:
test = learned_adj_matrix.view(17* 17)

In [32]:
edge_index, edge_weights = dense_to_sparse(learned_adj_matrix)

In [33]:
edge_weights

tensor([0.2522, 0.0987, 0.2695, 0.1731, 0.4706, 0.1125, 0.3106, 0.2640, 0.4164,
        0.4834, 0.2022, 0.0711, 0.0988, 0.3812, 0.4941, 0.4787, 0.3354, 0.4544,
        0.5590, 0.3079, 0.3901, 0.1605, 0.0505, 0.1401, 0.1620, 0.2574, 0.1982,
        0.3198, 0.2155, 0.0664, 0.2103, 0.5711, 0.2762, 0.0433, 0.3262, 0.5206,
        0.1117, 0.2694, 0.1328, 0.2747, 0.4183, 0.2704, 0.3227, 0.4283, 0.2009,
        0.2323, 0.0313, 0.2738, 0.1755, 0.3423, 0.1162, 0.1184, 0.0988, 0.4414,
        0.1709, 0.1148, 0.2762, 0.0911, 0.2768, 0.4247, 0.3289, 0.3534, 0.3554,
        0.5492, 0.1656, 0.0550, 0.4583, 0.2651, 0.1937, 0.2951, 0.3260, 0.1940,
        0.4359, 0.2285, 0.3260, 0.0206, 0.3546, 0.2014, 0.2235, 0.2543, 0.4745,
        0.1599, 0.2250, 0.0937, 0.2395, 0.1995, 0.2765, 0.2066, 0.3003, 0.1938,
        0.2983, 0.3402, 0.5992, 0.3475, 0.1838, 0.1235, 0.2920, 0.2451, 0.3256,
        0.2703, 0.1463, 0.1901, 0.5759, 0.1381, 0.2015, 0.0641, 0.2882, 0.1543,
        0.3026, 0.3461, 0.0848, 0.5388, 

In [ ]:
potential_causes

In [23]:
stats = pl.DataFrame(new_metadata)
agg_stats = stats.select(pl.sum("cause_1", "cause_2",'cause_3'))
agg_stats.select(pl.sum_horizontal(pl.all())).item()/stats.shape[0]

In [24]:
root_cause

'bfo2'

In [26]:
anomaly

'cso1'

In [27]:
np.mean(causal_graph)

In [25]:
adj_data

,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1
aimp,0.000000,0.206794,0.106287,0.114569,0.042025,0.171083,0.224217,0.145663,0.703853,0.205760,0.700846,0.232163,0.245771,0.537458,0.182045,0.109733,0.269548
amud,0.088947,0.000000,0.272621,0.100151,0.260866,0.146192,0.521083,0.426807,0.410943,0.421164,0.243287,0.334500,0.281939,0.256136,0.151344,0.561204,0.177459
arnd,0.088670,0.411724,0.000000,0.395733,0.107071,0.159322,0.462963,0.221731,0.095377,0.430168,0.229864,0.260246,0.373278,0.301274,0.369075,0.497310,0.513585
asin1,0.617530,0.114353,0.155496,0.000000,0.431426,0.051834,0.436027,0.379045,0.286911,0.595358,0.319157,0.429096,0.174969,0.411159,0.203245,0.330509,0.263160
asin2,0.310336,0.200686,0.318726,0.093504,0.000000,0.104311,0.367145,0.223119,0.366079,0.267407,0.118881,0.158229,0.509285,0.371102,0.086749,0.207700,0.174803
adbr,0.488893,0.186875,0.090075,0.358526,0.285870,0.000000,0.593338,0.176000,0.161949,0.331045,0.145740,0.318918,0.146959,0.122803,0.162105,0.603771,0.158054
adfl,0.248422,0.330216,0.152258,0.137409,0.494240,0.084427,0.000000,0.129814,0.098983,0.402223,0.188431,0.087488,0.176577,0.252805,0.107801,0.243595,0.443354
bed1,1.000000,1.000000,0.539573,0.502296,0.668211,0.715487,0.736942,0.000000,0.227470,0.292039,0.377602,0.387577,0.548140,0.267015,0.524470,0.721918,0.616672
bed2,0.666273,1.000000,0.664098,0.739226,0.623086,0.702429,0.402089,0.080537,0.000000,0.358047,0.331038,0.476915,0.498533,0.227834,0.520155,1.000000,0.664415
bfo1,0.430894,0.609346,1.000000,0.458554,0.558583,0.636146,1.000000,0.444974,0.155243,0.000000,0.356315,0.188026,0.129515,0.344933,0.729665,0.608088,1.000000


In [34]:
from sortedcontainers import SortedSet
import networkx as nx
import matplotlib.pyplot as plt
from dowhy import gcm 
import numpy as np
import pandas as pd


model.eval()
with torch.no_grad():
    _, _, _, learned_adj_matrix = model(data,ee,tt, num_nodes=num_nodes)
    causal_graph = learned_adj_matrix.cpu().numpy()

adj_matrix=causal_graph
threshold = np.mean(adj_matrix)
adj_matrix[adj_matrix < threshold] = 0
np.fill_diagonal(adj_matrix, 0, wrap=False)
new_data = mod_df
old_data = normal_df
potential_causes = potential_causes
cols = list(new_data.columns)
causal_graph = nx.DiGraph(adj_matrix)
def topological_remove_cycles(g):

    # Taken from: https://stackoverflow.com/questions/78348739/converting-cyclic-digraph-to-acyclic-digraph-dag
    # Dictionary of sets of incoming edges. We want to pick nodes with few of them.
    incoming = {}
    for node in g.nodes:
        incoming[node] = set()
    for node in g.nodes():
        for next_node in g.neighbors(node):
            incoming[next_node].add(node)

    # Sorted set of (count_incoming, -count_outgoing, node) triplets.
    # The first item in the set will have as few incoming nodes as it can, and as many outgoing.
    # In other words, it is a greedy choice for a good node to get rid of cycles.
    todo = SortedSet()
    for node, to_node in incoming.items():
        todo.add((len(to_node), -len(g.adj[node]), node))

    # And now let's start removing cycles.
    while 0 < len(todo):
        # Get the best node.
        _, _, node = todo.pop(0)
        to_node = incoming[node]
        for prev_node in to_node:
            # Each of these edges is in, or comes from, a cycle.
            if prev_node != node:
                # Do bookkeeping in todo.
                len_in = len(incoming[prev_node])
                len_out = len(g.adj[prev_node])
                todo.remove((len_in, -len_out, prev_node))
                todo.add((len_in, -len_out + 1, prev_node))
            g.remove_edge(prev_node, node)

        for next_node in g.neighbors(node):
            # Do bookkeeping in todo for forgetting about node.
            len_in = len(incoming[next_node])
            len_out = len(g.adj[next_node])
            todo.remove((len_in, -len_out, next_node))
            todo.add((len_in - 1, -len_out, next_node))
            incoming[next_node].remove(node)
        # And now we've guaranteed that node is cycle free and gone from our bookkeeping.


                
"""        nx.draw(self.causal_graph, with_labels=True, node_size=800, node_color="skyblue", 
            font_size=10, font_weight="bold", arrowsize=20)"""
            

'        nx.draw(self.causal_graph, with_labels=True, node_size=800, node_color="skyblue", \n            font_size=10, font_weight="bold", arrowsize=20)'

In [36]:
topological_remove_cycles(causal_graph) 

mapping = {}
for key, value in zip(list(causal_graph.nodes),cols):
    mapping[key] = value

causal_graph = nx.relabel_nodes(causal_graph, mapping)
causal_model = gcm.ProbabilisticCausalModel(causal_graph)
gcm.auto.assign_causal_mechanisms(causal_model, new_data,override_models=True)
gcm.fit(causal_model, new_data)

Fitting causal mechanism of node cso1: 100%|██████████| 17/17 [00:01<00:00, 13.45it/s]


In [37]:
causal_attributions = gcm.distribution_change(causal_model, old_data, new_data, anomaly)
abs_attributions = {}
for k, v in causal_attributions.items():
    if (k != anomaly) & (k in potential_causes):
        abs_attributions[k] = abs(v)

Evaluating set functions...: 100%|██████████| 295/295 [00:27<00:00, 10.73it/s]


In [38]:
causal_factors = sorted(abs_attributions.items(), key=lambda item: item[1], reverse=True)

In [39]:
causal_factors

[('bfo2', 1.8452791789060896),
 ('bed2', 0.33095667906828574),
 ('bfo1', 0.12611635945788793),
 ('bso1', 0.08249527988003397),
 ('bso2', 0.06529990137770968),
 ('bso3', 0.019364852287029916),
 ('bed1', 0.003613218136100902)]

In [40]:
root_cause

'bfo2'

In [41]:
att = gcm.attribute_anomalies(causal_model,target_node=anomaly,anomaly_samples=new_data)

Evaluating set functions...:  65%|██████▍   | 192/297 [12:04<07:21,  4.20s/it]

KeyboardInterrupt: 